### Deep Optimal Stopping - Dice problem - Implementation with tensorflow estimator 

#### Work is in progress... updates are expected

In [1]:
import tensorflow as tf
import numpy as np 
import scipy
tf.__version__

'1.8.0'

In [2]:
# define sample size for training 
M = 10000

# create a sample of M x 3
# In this example we only consider 3 tosses, hence only two steps with choices to stop at
dice = np.random.randint(low=1, high=7, size=(M, 3))

In [3]:
# see the first 10 paths from the samples generated above
print("The first 10 samples: ")
print(dice[:10, :])

The first 10 samples: 
[[1 3 3]
 [3 6 5]
 [2 1 3]
 [5 3 2]
 [3 2 4]
 [1 1 2]
 [5 4 5]
 [3 2 6]
 [4 6 3]
 [2 6 6]]


In [4]:
# define input function for training with estimator, and use the dice np dataset 
def numpy_train_input_fn(dice): 
    return tf.estimator.inputs.numpy_input_fn(
        x={"t0": np.reshape(dice[:, 0].astype(float), (len(dice[:, 0]), 1)), 
           "t1": np.reshape(dice[:, 1].astype(float), (len(dice[:, 1]), 1)), 
           "t2": np.reshape(dice[:, 2].astype(float), (len(dice[:, 2]), 1))},
        batch_size = 64, 
        num_epochs = 30, 
        shuffle = True, 
        queue_capacity = 1000
    )

# define input function for evaluation
def numpy_eval_input_fn(dice):
    return tf.estimator.inputs.numpy_input_fn(
        x={"t0": np.reshape(dice[:, 0].astype(float), (len(dice[:, 0]), 1)), 
           "t1": np.reshape(dice[:, 1].astype(float), (len(dice[:, 1]), 1)), 
           "t2": np.reshape(dice[:, 2].astype(float), (len(dice[:, 2]), 1))},
        num_epochs = 1, 
        shuffle = False
    )

In [5]:
def my_model_fn(features, mode, params):  
    
    """Defining the custon architecture"""
    
    input_set = tf.concat([features['t0'], features['t1'], features['t2']], 1, name="input_set")
    
    # Step 1 - Configure the network 
    with tf.variable_scope("2nd", reuse=tf.AUTO_REUSE):
        
        first_hidden_layer_2nd = tf.layers.dense(features['t1'], 51, activation=tf.nn.relu,
                                            #kernel_initializer=tf.glorot_normal_initializer(),
                                            kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.001), 
                                            kernel_regularizer= tf.contrib.layers.l2_regularizer(scale=0.3),
                                            name="first_layer_2nd")
        second_hidden_layer_2nd = tf.layers.dense(first_hidden_layer_2nd, 51, activation=tf.nn.relu, 
                                            #kernel_initializer=tf.glorot_normal_initializer(),
                                            kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.001),
                                            kernel_regularizer = tf.contrib.layers.l2_regularizer(scale=0.3), 
                                            name="second_layer_2nd")
        output_layer_2nd_na = tf.layers.dense(second_hidden_layer_2nd, 1, activation=None, 
                                          #kernel_initializer=tf.glorot_normal_initializer(),
                                          kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.001),
                                            kernel_regularizer = tf.contrib.layers.l2_regularizer(scale=0.3),
                                          name="output_layer_2nd_na")
        output_layer_2nd = tf.layers.dense(second_hidden_layer_2nd, 1, activation=tf.nn.sigmoid, 
                                          #kernel_initializer=tf.glorot_normal_initializer(),
                                          kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.001),
                                            kernel_regularizer = tf.contrib.layers.l2_regularizer(scale=0.3),
                                          name="output_layer_2nd")
        
        #tau_1 = 1*output_layer_2nd_ind + 2*(1-output_layer_2nd_ind)
        
        # Define reward and optimizer
        one = tf.constant(1, dtype=tf.float64)
        reward_2nd = tf.add(tf.multiply(output_layer_2nd, features['t1']), 
                        tf.multiply((one-output_layer_2nd), features['t2']), 
                       name = "reward_2nd")
    
        rAvg_2nd = tf.reduce_mean(reward_2nd) 
        cost_2nd = tf.scalar_mul(-1,rAvg_2nd)
        op2 = tf.train.AdamOptimizer(learning_rate=0.0001)
    
        
        second_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="2nd")
        train_op2 = op2.minimize(cost_2nd, var_list=second_train_vars)
    
    
    with tf.variable_scope("1st", reuse=tf.AUTO_REUSE):
        
        first_hidden_layer_1st = tf.layers.dense(features['t0'], 51, activation=tf.nn.relu,
                                            kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.001),
                                            kernel_regularizer = tf.contrib.layers.l2_regularizer(scale=0.3), 
                                            #kernel_initializer=tf.glorot_normal_initializer(),
                                            name="first_layer_1st")
        second_hidden_layer_1st = tf.layers.dense(first_hidden_layer_1st, 51, activation=tf.nn.relu, 
                                            kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.001), 
                                            kernel_regularizer = tf.contrib.layers.l2_regularizer(scale=0.3),
                                            #kernel_initializer=tf.glorot_normal_initializer(),
                                            name="second_layer_1st")
        
        output_layer_1st_na = tf.layers.dense(second_hidden_layer_1st, 1, activation=None,
                                          kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.001),
                                            kernel_regularizer = tf.contrib.layers.l2_regularizer(scale=0.3),
                                          #kernel_initializer=tf.glorot_normal_initializer(),
                                          name="output_layer_1st_na")
        output_layer_1st = tf.layers.dense(second_hidden_layer_1st, 1, activation=tf.nn.sigmoid, 
                                          kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.001),
                                            kernel_regularizer = tf.contrib.layers.l2_regularizer(scale=0.3),
                                          #kernel_initializer=tf.glorot_normal_initializer(),
                                          name="output_layer_1st")
        
    
    with tf.get_default_graph().control_dependencies([train_op2]):
        output_layer_2nd_ind = tf.cast(tf.clip_by_value(tf.sign(output_layer_2nd_na), 0, 2), 
                                       dtype=tf.int32, name="output_layer_2nd_ind")
        
        tau_1 = 1*output_layer_2nd_ind + 2*(1-output_layer_2nd_ind)
        
        col_1 = tf.reshape(tf.range(tf.shape(tau_1)[0]), shape=tf.shape(tau_1), name="col_1")
        indices_1 = tf.concat([col_1, tau_1], 1, name="indices_1")
        g_1 = tf.gather_nd(input_set, indices=indices_1, name="g_1")
    
        reward_1st = tf.add(tf.multiply(output_layer_1st, features['t0']), 
                            tf.multiply((one-output_layer_1st), g_1), 
                            name = "reward_1st")
        rAvg_1st = tf.reduce_mean(reward_1st)
        cost_1st = tf.scalar_mul(-1,rAvg_1st)
    
        op1 = tf.train.AdamOptimizer(learning_rate=0.0001)
        
        first_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="1st")
        train_op1 = op1.minimize(cost_1st, var_list=first_train_vars)
    
    
    with tf.get_default_graph().control_dependencies([train_op2, train_op1]):
        output_layer_1st_ind = tf.cast(tf.clip_by_value(tf.sign(output_layer_1st_na), 0, 2), 
                                       dtype=tf.int32, name="output_layer_1st_ind")
        
        
        tau_0 = (0*output_layer_1st_ind + 1*output_layer_2nd_ind*(1-output_layer_1st_ind)
                +2*(1-output_layer_1st_ind)*(1-output_layer_2nd_ind))
        indices_0 = tf.concat([col_1, tau_0], 1, name="indices_0")
        g_0 = tf.gather_nd(input_set, indices=indices_0, name="g_0")
        
    
        price = tf.reduce_mean(g_0, name="price")
    
    global_step = tf.train.get_global_step()
    update_global_step = tf.assign(global_step, global_step + 1, name = 'update_global_step')
    
    train_op =tf.group(train_op1, train_op2)
    cost = cost_1st+cost_2nd 
    
    op_ = tf.train.AdamOptimizer(learning_rate=0.005)
    train_op_ =op_.minimize(cost)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            loss=price,
            evaluation_hooks=None)
       
    
    # Provide an estimator spec for `ModeKeys.PREDICT`.
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"price": price})
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(
            mode=mode, 
            loss=cost_1st, 
            train_op=tf.group(train_op, update_global_step))


In [6]:
nn = tf.estimator.Estimator(model_fn=my_model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/16/6vfzqvh50sv0n670v2qmktsw0000gn/T/tmplsusxzqo', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11c381f28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
nn.train(input_fn=numpy_train_input_fn(dice), steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/16/6vfzqvh50sv0n670v2qmktsw0000gn/T/tmplsusxzqo/model.ckpt.
INFO:tensorflow:loss = -3.5703124811997005, step = 1
INFO:tensorflow:global_step/sec: 163.59
INFO:tensorflow:loss = -3.468976812358656, step = 101 (0.611 sec)
INFO:tensorflow:global_step/sec: 438.079
INFO:tensorflow:loss = -3.7000767361728792, step = 201 (0.229 sec)
INFO:tensorflow:global_step/sec: 455.355
INFO:tensorflow:loss = -3.377028461643883, step = 301 (0.219 sec)
INFO:tensorflow:global_step/sec: 470.954
INFO:tensorflow:loss = -3.724929109447869, step = 401 (0.212 sec)
INFO:tensorflow:global_step/sec: 461.189
INFO:tensorflow:loss = -3.89612278458802, step = 501 (0.217 sec)
INFO:tensorflow:global_step/sec: 354.491
INFO:tensorflow:

In [8]:
ev = nn.evaluate(input_fn=numpy_eval_input_fn(dice))
print("Loss: %s" % ev["loss"])


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-27-14:03:58
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/16/6vfzqvh50sv0n670v2qmktsw0000gn/T/tmplsusxzqo/model.ckpt-4689
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-27-14:03:59
INFO:tensorflow:Saving dict for global step 4689: global_step = 4689, loss = 2.8472111
Loss: 2.8472111


In [9]:
dice_eval = np.random.randint(low=1, high=7, size=(M, 3))

In [10]:
ev_ck = nn.evaluate(input_fn=numpy_eval_input_fn(dice_eval))
print("Loss: %s" % ev_ck["loss"])


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-27-14:04:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/16/6vfzqvh50sv0n670v2qmktsw0000gn/T/tmplsusxzqo/model.ckpt-4689
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-27-14:04:12
INFO:tensorflow:Saving dict for global step 4689: global_step = 4689, loss = 2.815368
Loss: 2.815368


In [ ]:
#def predict_input_fn(dice_eval):
#    return tf.estimator.inputs.numpy_input_fn(
#         x={"t0": np.reshape(dice[:, 0].astype(float), (len(dice[:, 0]), 1)), 
#           "t1": np.reshape(dice[:, 1].astype(float), (len(dice[:, 1]), 1)), 
#           "t2": np.reshape(dice[:, 2].astype(float), (len(dice[:, 2]), 1))}, 
#        num_epochs = 1, 
#        shuffle = False)


